In [ ]:
"""
Created on Mon Jul 24 11:36 2020

This is a script to cut out the T and S in the 50 km in front of the ice front

@author: Clara Burgard
"""

- calculate the distance to the ice front for the small domain in front of the ice shelf
- take the ocean points at distance of ~50 km of the ice front 

In [ ]:
import xarray as xr
import numpy as np
import pandas as pd
#from tqdm.notebook import tqdm
from tqdm import tqdm
import gsw
import matplotlib.pyplot as plt
import basal_melt_param.useful_functions as uf
import basal_melt_param.T_S_profile_functions as tspf
import basal_melt_param.melt_functions as meltf
import basal_melt_param.box_functions as bf

from scipy.spatial import cKDTree


import itertools

import distributed
import glob

In [ ]:
%matplotlib qt5

In [ ]:
client = distributed.Client(n_workers=4, dashboard_address=':8795', local_directory='/tmp', memory_limit='6GB')

In [ ]:
client

In [ ]:
%matplotlib qt5

READ IN THE DATA

In [ ]:
CMIP_run = 'CNRM-CM6-1' # 'EPM026','EPM031', 'EPM034'
scenario = 'historical'

if CMIP_run == 'CNRM-CM6-1':
    ens_run = 'r1i1p1f2'

In [ ]:
inputpath_data_orig='/bettik/jourdai1/OCEAN_DATA_CMIP6_STEREO/'+CMIP_run+'/'
inputpath_profiles='/bettik/burgardc/DATA/SUMMER_PAPER/interim/T_S_PROF/cmip_4km_'+CMIP_run+'/'
inputpath_isf='/bettik/burgardc/DATA/SUMMER_PAPER/interim/ANTARCTICA_IS_MASKS/BedMachine_8km/'
inputpath_BedMachine='/bettik/burgardc/DATA/SUMMER_PAPER/interim/'

# make the domain a little smaller to make the computation even more efficient - file isf has already been made smaller at its creation
map_lim = [-3000000,3000000]

COMPUTING THE MEAN PROFILES

CONTINENTAL SHELF

In [ ]:
mask_domain_distkm = 50000

If workers don't die (with 12 cores, took approx 1hour), if workers die, divide work by years

In [ ]:
all_in_one = True # False if worker die, True if workers don't die
if all_in_one:
    dist_to_front_file = xr.open_mfdataset(inputpath_profiles+'dist_to_ice_front_only_contshelf.nc',chunks={'x': 50, 'y': 50})
    T_ocean_files = xr.open_mfdataset(inputpath_data_orig+'thetao_Oyr_'+mod+'_'+scenario+'_'+ens_run+'_*.nc', chunks={'x': 50, 'y': 50, 'depth': 50, 'time': 5}, parallel=True)
    S_ocean_files = xr.open_mfdataset(inputpath_data_orig+'so_Oyr_'+mod+'_'+scenario+'_'+ens_run+'_*.nc', chunks={'x': 50, 'y': 50, 'depth': 50, 'time': 5}, parallel=True)

    #T_S_ocean_1980 = xr.open_mfdataset(inputpath_profiles+'T_S_theta_ocean_corrected_1990.nc',chunks={'x': 50, 'y': 50, 'depth': 50})
    #T_S_ocean_1980 = xr.open_mfdataset(inputpath_profiles+'T_S_theta_ocean_corrected_'+str(yy_ex)+'.nc',chunks={'x': 50, 'y': 50, 'depth': 50})

dist_to_front = dist_to_front_file['dist_from_front']

In [ ]:
dist_to_front_file_yy = xr.open_dataset(inputpath_profiles+'dist_to_ice_front_only_contshelf.nc').chunk({'Nisf': 5})
dist_to_front = dist_to_front_file_yy['dist_from_front']
mask_km = dist_to_front <= mask_domain_distkm

#for yy in tqdm(range(yy_start,yy_end+1)):
for yy in tqdm(range(2077,yy_end+1)):
    T_S_ocean_file_yy = xr.open_dataset(inputpath_profiles+'T_S_theta_ocean_corrected_'+str(yy)+'.nc').chunk({'depth': 5})
    
    ds_sum = (T_S_ocean_file_yy * mask_km).sum(['x','y'])
    
    mask_depth = T_S_ocean_file_yy['salinity_ocean'].squeeze().drop('time') > 0
    mask_all = mask_km & mask_depth
    
    mask_sum = mask_all.sum(['x','y'])
    mask_sum = mask_sum.load()
    
    ds_mean = ds_sum/mask_sum
    ds_mean.to_netcdf(inputpath_profiles+'T_S_mean_prof_corrected_km_contshelf_'+str(yy)+'.nc')

In [ ]:
TS_list = []
for tt in range(yy_start,yy_end+1):
    ds_tt = xr.open_dataset(inputpath_profiles+'T_S_mean_prof_corrected_km_contshelf_'+str(tt)+'.nc')
    TS_list.append(ds_tt)
TS_ds = xr.concat(TS_list, dim='time')
TS_ds.to_netcdf(inputpath_profiles+'T_S_mean_prof_corrected_km_contshelf_allyy.nc')